# Analyze NBI bridges for SPC planning region

In [1]:
import pandas as pd

### Import 2012-22 NBI data for Pennsylvania

In [2]:
columns = [
    'COUNTY_CODE_003',
    'STRUCTURE_NUMBER_008',
    'OWNER_022',
    'FUNCTIONAL_CLASS_026',
    'DECK_COND_058',
    'SUPERSTRUCTURE_COND_059',
    'SUBSTRUCTURE_COND_060',
    'CULVERT_COND_062'
]
years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
years_nbi = []

for year in years:
    year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=columns)
    year_nbi['year'] = year
    years_nbi.append(year_nbi)

nbi = pd.concat(years_nbi)
nbi['STRUCTURE_NUMBER_008'] = nbi.STRUCTURE_NUMBER_008.astype(str).str.zfill(15)

/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_60839/1871938710.py:15: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=columns)
/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_60839/1871938710.py:15: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=columns)
/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_60839/1871938710.py:15: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=columns)
/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_60839/1871938710.py:15: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  year_nbi = pd.read_csv('input/nbi%s.csv' % year, usecols=colum

In [3]:
len(nbi.STRUCTURE_NUMBER_008.unique())

25941

In [4]:
nbi.head(1)

,STRUCTURE_NUMBER_008,COUNTY_CODE_003,OWNER_022,FUNCTIONAL_CLASS_026,DECK_COND_058,SUPERSTRUCTURE_COND_059,SUBSTRUCTURE_COND_060,CULVERT_COND_062,year
0,1PA0099,41.0,73,9,N,N,N,8,2012


### Limit to SPC planning region

In [5]:
nbi = nbi[nbi.COUNTY_CODE_003.isin([3, 5, 7, 19, 51, 59, 63, 73, 125, 129])]

In [6]:
len(nbi.STRUCTURE_NUMBER_008.unique())

5498

### Limit to bridges owned by PennDOT, counties or municipalities

In [7]:
nbi = nbi[nbi.OWNER_022.isin([1, 2, 3, 4])]

In [8]:
len(nbi.STRUCTURE_NUMBER_008.unique())

5083

### Introduce custom condition field

In [9]:
nbi['BridgeConditionNum'] = nbi[[
    'DECK_COND_058',
    'SUPERSTRUCTURE_COND_059',
    'SUBSTRUCTURE_COND_060',
    'CULVERT_COND_062'
]].min(axis=1)

nbi.loc[nbi.BridgeConditionNum == 'N', 'BridgeConditionNum'] = -100
nbi['BridgeConditionNum'] = nbi.BridgeConditionNum.astype(int)

nbi.loc[nbi.BridgeConditionNum >= 7, 'BridgeCondition'] = 'G'
nbi.loc[nbi.BridgeConditionNum.isin([5, 6]), 'BridgeCondition'] = 'F'
nbi.loc[nbi.BridgeConditionNum <= 4, 'BridgeCondition'] = 'P'
nbi.loc[nbi.BridgeConditionNum == -100, 'BridgeCondition'] = 'N/A'

### Introduce custom owner field

In [10]:
nbi['SimpleOwner'] = nbi.OWNER_022

In [11]:
nbi['SimpleOwner'] = nbi.SimpleOwner.replace({
    1: 'PennDOT',
    2: 'County',
    3: 'Municipal',
    4: 'Municipal'
})

### Look at all bridges by owner, condition

In [12]:
piv = pd.pivot_table(nbi[['year', 'SimpleOwner', 'BridgeCondition']], index=['year', 'SimpleOwner'], columns='BridgeCondition', values='BridgeCondition', aggfunc='size')
piv['Total'] = piv.sum(axis=1)

In [13]:
piv['P%'] = 100 * (piv.P / piv.Total)
piv['P%_total'] = 100 * (piv.P / piv.groupby('year').P.sum())

In [14]:
piv

BridgeCondition        F       G  N/A      P   Total         P%   P%_total
year SimpleOwner                                                          
2012 County        285.0   160.0  NaN  235.0   680.0  34.558824  18.680445
     Municipal     253.0   147.0  NaN  182.0   582.0  31.271478  14.467409
     PennDOT      1620.0   650.0  NaN  841.0  3111.0  27.033108  66.852146
2013 County        288.0   143.0  NaN  240.0   671.0  35.767511  20.565553
     Municipal     257.0   141.0  NaN  176.0   574.0  30.662021  15.081405
     PennDOT      1659.0   693.0  NaN  751.0  3103.0  24.202385  64.353042
2014 County        293.0   140.0  1.0  233.0   667.0  34.932534  21.297989
     Municipal     264.0   150.0  NaN  174.0   588.0  29.591837  15.904936
     PennDOT      1675.0   729.0  NaN  687.0  3091.0  22.225817  62.797075
2015 County        301.0   147.0  NaN  225.0   673.0  33.432392  21.047708
     Municipal     274.0   142.0  NaN  202.0   618.0  32.686084  18.896165
     PennDOT      1677.0   777.0  NaN  642.0  3096.0  20.736434  60.056127
2016 County        310.0   167.0  NaN  198.0   675.0  29.333333  20.183486
     Municipal     294.0   151.0  NaN  200.0   645.0  31.007752  20.387360
     PennDOT      1678.0   822.0  NaN  583.0  3083.0  18.910152  59.429154
2017 County        315.0   181.0  NaN  182.0   678.0  26.843658  19.825708
     Municipal     296.0   152.0  NaN  201.0   649.0  30.970724  21.895425
     PennDOT      1659.0   891.0  NaN  535.0  3085.0  17.341977  58.278867
2018 County        320.0   190.0  NaN  167.0   677.0  24.667651  20.848939
     Municipal     304.0   148.0  NaN  197.0   649.0  30.354391  24.594257
     PennDOT      1659.0   956.0  NaN  437.0  3052.0  14.318480  54.556804
2019 County        328.0   195.0  NaN  153.0   676.0  22.633136  21.763869
     Municipal     318.0   135.0  NaN  196.0   649.0  30.200308  27.880512
     PennDOT      1673.0  1049.0  NaN  354.0  3076.0  11.508453  50.355619
2020 County        332.0   195.0  NaN  150.0   677.0  22.156573  23.219814
     Municipal     321.0   134.0  NaN  195.0   650.0  30.000000  30.185759
     PennDOT      1677.0  1102.0  NaN  301.0  3080.0   9.772727  46.594427
2021 County        344.0   195.0  NaN  143.0   682.0  20.967742  22.916667
     Municipal     319.0   144.0  NaN  200.0   663.0  30.165913  32.051282
     PennDOT      1670.0  1136.0  NaN  281.0  3087.0   9.102689  45.032051
2022 County        341.0   199.0  NaN  133.0   673.0  19.762259  21.983471
     Municipal     328.0   139.0  NaN  201.0   668.0  30.089820  33.223140
     PennDOT      1677.0  1144.0  NaN  271.0  3092.0   8.764554  44.793388

##### Export summary for Datawrapper

In [15]:
piv_dw = piv[['P%']].reset_index()
pd.pivot_table(piv_dw, index='year', columns='SimpleOwner', values='P%').to_csv('output/poorpct.csv')

##### Export latest year's NBI data for construction analysis

In [16]:
nbi[nbi.year == '2022'][[
    'STRUCTURE_NUMBER_008',
    'BridgeCondition',
    'SimpleOwner'
]].to_csv('output/nbi2022-formatted.csv', index=False)

### Look at off-system bridges by owner, condition

In [17]:
piv = pd.pivot_table(nbi[nbi.FUNCTIONAL_CLASS_026.isin([8, 9, 19])][['year', 'SimpleOwner', 'BridgeCondition']], index=['year', 'SimpleOwner'], columns='BridgeCondition', values='BridgeCondition', aggfunc='size')
piv['Total'] = piv.sum(axis=1)

In [18]:
piv['P%'] = 100 * (piv.P / piv.Total)
piv['P%_total'] = 100 * (piv.P / piv.groupby('year').P.sum())

In [19]:
piv

BridgeCondition     F    G    P  Total         P%   P%_total
year SimpleOwner                                            
2012 County       201  123  159    483  32.919255  25.521669
     Municipal    227  132  162    521  31.094050  26.003210
     PennDOT      370  138  302    810  37.283951  48.475120
2013 County       197  117  159    473  33.615222  26.677852
     Municipal    229  127  156    512  30.468750  26.174497
     PennDOT      380  145  281    806  34.863524  47.147651
2014 County       205  115  152    472  32.203390  26.116838
     Municipal    236  135  154    525  29.333333  26.460481
     PennDOT      371  157  276    804  34.328358  47.422680
2015 County       216  115  145    476  30.462185  24.369748
     Municipal    252  128  182    562  32.384342  30.588235
     PennDOT      369  163  268    800  33.500000  45.042017
2016 County       221  123  139    483  28.778468  24.733096
     Municipal    270  138  177    585  30.256410  31.494662
     PennDOT      365  166  246    777  31.660232  43.772242
2017 County       222  130  132    484  27.272727  24.535316
     Municipal    272  139  179    590  30.338983  33.271375
     PennDOT      357  184  227    768  29.557292  42.193309
2018 County       228  140  120    488  24.590164  25.052192
     Municipal    280  135  175    590  29.661017  36.534447
     PennDOT      357  220  184    761  24.178712  38.413361
2019 County       234  145  108    487  22.176591  24.942263
     Municipal    293  124  175    592  29.560811  40.415704
     PennDOT      364  260  150    774  19.379845  34.642032
2020 County       239  144  105    488  21.516393  25.798526
     Municipal    296  123  174    593  29.342327  42.751843
     PennDOT      360  279  128    767  16.688396  31.449631
2021 County       248  145  101    494  20.445344  24.694377
     Municipal    295  132  178    605  29.421488  43.520782
     PennDOT      353  288  130    771  16.861219  31.784841
2022 County       247  148   94    489  19.222904  23.558897
     Municipal    303  127  180    610  29.508197  45.112782
     PennDOT      351  292  125    768  16.276042  31.328321

# Analyze formerly poor NBI bridges for SPC planning region

In [20]:
formerlypoor = pd.concat([
    pd.read_csv('input/PA_Poor12_to_Fair22.csv', quotechar="'"),
    pd.read_csv('input/PA_Poor12_to_Good22.csv', quotechar="'"),
])

In [21]:
formerlypoor['8 - Structure Number'] = formerlypoor['8 - Structure Number'].astype(str).str.zfill(15)

In [22]:
formerlypoor = pd.merge(
    formerlypoor[['8 - Structure Number']],
    nbi[nbi.year == '2022'],
    left_on='8 - Structure Number',
    right_on='STRUCTURE_NUMBER_008'
)

formerlypoor['SimpleOwner'] = formerlypoor.OWNER_022
formerlypoor['SimpleOwner'] = formerlypoor.SimpleOwner.replace({
    1: 'PennDOT',
    2: 'County',
    3: 'Municipal',
    4: 'Municipal'
})

In [23]:
formerlypoor.SimpleOwner.value_counts()

PennDOT      228
County        79
Municipal     35
Name: SimpleOwner, dtype: int64